In [16]:
#https://bergvca.github.io/2017/10/14/super-fast-string-matching.html
import time
import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix , vstack

names =  pd.read_excel('RIR from server.xlsx')
print('The shape: %d x %d' % names.shape)
names.head()

The shape: 3984 x 4


,Eorp Id,Knox Pf Id,Potential Failure,LastDate
0,NaN,NaN,1. Delay/inaccurate interpretation of importan...,2017-11-30
1,NaN,NaN,1. Failure to comply with requirements from UK...,2018-09-30
2,NaN,NaN,1. Failure to comply with requirements from UK...,2018-09-30
3,NaN,NaN,1. Failure to consider requirements from UK re...,2018-09-30
4,NaN,NaN,1. Failure to incorporate the key risk drivers...,2018-09-30


In [3]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "McDonalds":')
ngrams('McDonalds')

All 3-grams in "McDonalds":


['McD', 'cDo', 'Don', 'ona', 'nal', 'ald', 'lds']

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

company_names = names['Potential Failure'].fillna("CVxTz").values
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)

In [7]:
import pyodbc

conn = pyodbc.connect('DRIVER={SQL Server};SERVER=#####;DATABASE=#####;UID=####;PWD=#####')
stmt_PF = """select distinct PotentialFailureID, PotentialFailureName
            , min(cob) as FirstDate, max(cob) as LastDate 
            from src_EORP_ControltoPF
            where PotentialFailureName is not null
            group by PotentialFailureID, PotentialFailureName """
         
#print stmt_complinet

PF = pd.read_sql(stmt_PF,conn)

lookup_names=PF.PotentialFailureName
lookup_term_mat=vectorizer.transform(lookup_names)

In [33]:
def get_matches_df(sparse_matrix, name_vector, lookup_vector):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = lookup_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [9]:
def batch_cosine(A, B, lower_bound=0):
    x,y=A.shape
    batch_size=200
    loop=int(x/batch_size)
    #loop=3
    for i in range(loop +1 ):
        if i%10==0:
            print('Working on ',i*batch_size, 'to ',min((i+1)*batch_size,x))
            structTime = time.localtime()
            print(datetime.datetime(*structTime[:6]))

        out=cosine_similarity(A[i*batch_size:min((i+1)*batch_size,x)],B)
        out[out<lower_bound]=0
        out_mat=csr_matrix(out)

        if i==0:
            matches=out_mat
        else:
            matches=vstack([matches,out_mat])
    return matches

In [37]:
matches=batch_cosine(lookup_term_mat,tf_idf_matrix,lower_bound=0.7)

Working on  0 to  200
2018-12-11 11:34:02
Working on  2000 to  2200
2018-12-11 11:34:04


In [35]:
lookup_names

0                     Accounting Controls Actual vs Flash
1              Accounting Controls Balance Substantiation
2       Adverse case outcome resulting from incorrect ...
3       Annual Board and Committees Effectiveness Reviews
4                                 Annual Sanctions Review
5       Approval and release of a stock exchange annou...
6                        Arrange, Hold and Minute meeting
7       B&M | Brand Design & Planning | Contract Exit ...
8       B&M | Brand Design & Planning | Contract Legal...
9       B&M | Brand Design & Planning | Group Approval...
10      B&M | Brand Design & Planning | Information Se...
11      B&M | Brand Design & Planning | Name screening...
12      B&M | Brand Design & Planning | Name Screening...
13      B&M | Brand Management | Contract Exit Conditi...
14      B&M | Brand Management | Contract Legal Enforc...
15        B&M | Brand Management | Cross Border HLRS | PF
16      B&M | Brand Management | Group Approval on Fin...
17      B&M | 

In [46]:

matches_df = get_matches_df(matches,lookup_names,company_names)
matches_df.to_csv('RIR_out.csv')

In [45]:
print(matches_df.left_side[17])
print(matches_df.right_side[17])

Cash | Product Management | Inappropriate  and incorrect pricing for TB products
Inappropriate pricing for TB products
